In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pickle
import re
import csv
from torch.utils.data import DataLoader, TensorDataset
import ast
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import StandardScaler
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
import os

In [2]:
# (MultiHeadAttention, EncoderLayer 클래스는 기존 코드와 동일하게 사용)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads

        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        self.W_O = nn.Linear(d_model, d_model)
 
    def forward(self, Q, K, V):
        Q = self.W_Q(Q)
        K = self.W_K(K)
        V = self.W_V(V)

        Q = self._split_heads(Q)
        K = self._split_heads(K)
        V = self._split_heads(V)

        attention_weights = torch.matmul(Q, K.transpose(-1, -2)) / torch.sqrt(torch.tensor(self.depth, dtype=torch.float32))
        attention_weights = torch.softmax(attention_weights, dim=-1)

        output = torch.matmul(attention_weights, V)
        output = self._combine_heads(output)

        output = self.W_O(output)
        return output
 
    def _split_heads(self, tensor):
        tensor = tensor.view(tensor.size(0), -1, self.num_heads, self.depth)
        return tensor.transpose(1, 2)
 
    def _combine_heads(self, tensor):
        tensor = tensor.transpose(1, 2).contiguous()
        return tensor.view(tensor.size(0), -1, self.num_heads * self.depth)


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.feedforward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
 
    def forward(self, x):
        attention_output = self.attention(x, x, x)
        attention_output = self.norm1(x + attention_output)

        feedforward_output = self.feedforward(attention_output)
        output = self.norm2(attention_output + feedforward_output)
        return output

# [!!! 모델 수정 !!!]
class iTransformer(nn.Module):
    """
    iTransformer (Encoder-Only)
    - input_len (seq_len): 입력 시퀀스 길이 (예: 1024)
    - output_len (pred_len): 예측 시퀀스 길이 (예: 14)
    - num_features (n_vars): 변수 개수 (예: 5)
    - hidden_dim (d_model): 모델의 임베딩 차원 (예: 64)
    """
    def __init__(self, input_len, output_len, num_features, hidden_dim, num_heads, num_layers):
        super(iTransformer, self).__init__()
        
        # [수정] 입력 레이어: (Batch, N, L) -> (Batch, N, d_model)
        # L = input_len, N = num_features
        self.input_layer = nn.Linear(input_len, hidden_dim)
        
        # [추가] 변수(토큰)를 위한 Positional Embedding
        self.pos_embedding = nn.Parameter(torch.zeros(1, num_features, hidden_dim))
        
        # 인코더 레이어 (기존과 동일)
        self.encoder_layers = nn.ModuleList([EncoderLayer(hidden_dim, num_heads) for _ in range(num_layers)])
        
        # [수정] 출력 레이어: (Batch, N, d_model) -> (Batch, N, P)
        # P = output_len
        self.output_layer = nn.Linear(hidden_dim, output_len)
 
    def forward(self, x):
        # x의 입력 형태: (Batch, L, N) = (Batch, 1024, 5)
        
        # 1. 전치 (Transpose): (B, L, N) -> (B, N, L)
        # 변수(N)를 토큰(시퀀스) 차원으로, 시간(L)을 임베딩 차원으로
        x = x.permute(0, 2, 1)
        
        # 2. 임베딩: (B, N, L) -> (B, N, d_model)
        x = self.input_layer(x)
        
        # 3. Positional Embedding 추가
        x = x + self.pos_embedding

        # 4. 인코더 (어텐션은 N x N (5x5) 차원에서 수행됨)
        encoder_output = x
        for layer in self.encoder_layers:
            encoder_output = layer(encoder_output)

        # 5. 출력 레이어 (프로젝션): (B, N, d_model) -> (B, N, P)
        decoder_output = self.output_layer(encoder_output)
        
        # 6. 최종 전치 (Transpose): (B, N, P) -> (B, P, N)
        # (Batch, Pred_Len, Num_Features) 형태로 변환
        output = decoder_output.permute(0, 2, 1)
        
        return output

In [3]:
csv_files = [
    'BROADCOM 5년치.csv', 'ALPHABET C 5년치.csv', 'AMAZON 5년치.csv', 
    'APPLE 5년치.csv', 'META 5년치.csv', 'MICROSOFT 5년치.csv', 
    'NETFLIX 5년치.csv', 'NVIDIA 5년치.csv', 'PALANTIR 5년치.csv', 'TESLA 5년치.csv'
]
feature_cols = ['Close/Last', 'Volume', 'Open', 'High', 'Low']

# [수정] num_features = 10개 주식 * 5개 변수 = 50개
num_features = len(csv_files) * len(feature_cols) 

input_len = 1024
output_len = 14
hidden_dim = 128 # (50개 변수를 다루므로 128로 늘려도 좋습니다)
num_heads = 8
num_layers = 6
num_epochs = 100
batch_size = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# --- 1. [수정] 10개 주식 데이터를 하나의 (Total_Len, 50) DataFrame으로 통합 ---
print("--- 모든 주식 데이터 통합 ---")
all_data_dfs = []
column_names = [] # 50개 컬럼 이름 저장

for csv_file in csv_files:
    stock_name = re.match(r'^[A-Z]+', csv_file).group(0)
    
    try:
        df = pd.read_csv(csv_file, parse_dates=['Date'], index_col='Date')
        df = df.sort_index(ascending=True)
        
        # 클리닝
        cleaned_cols = {}
        for col in feature_cols:
            series = df[col]
            if series.dtype == 'object':
                series = series.str.replace(r'[$,]', '', regex=True)
            series = pd.to_numeric(series, errors='coerce')
            
            # 컬럼 이름 변경 (예: APPLE_Close/Last, APPLE_Volume)
            new_col_name = f"{stock_name}_{col}"
            cleaned_cols[new_col_name] = series
            column_names.append(new_col_name)
            
        all_data_dfs.append(pd.DataFrame(cleaned_cols))
        
    except Exception as e:
        print(f"오류: {csv_file} 처리 중 - {e}")

# Date 인덱스 기준으로 모든 DataFrame을 옆으로(axis=1) 합침
all_data_wide_df = pd.concat(all_data_dfs, axis=1)

# [중요] 모든 주식이 상장되어 데이터가 있는 날짜만 사용 (NaN 행 제거)
all_data_wide_df = all_data_wide_df.dropna()

all_data_np = all_data_wide_df.to_numpy(dtype=np.float32)
print(f"통합 데이터 형태: {all_data_np.shape}") # (N_days, 50)

if len(all_data_np) < input_len + output_len:
    raise ValueError("데이터 부족: 모든 주식이 동시에 상장된 기간이 너무 짧습니다.")

# --- 2. [수정] 단일 스케일러 훈련 및 저장 ---
print("--- 단일 통합 스케일러 훈련 ---")
scaler = StandardScaler()
scaled_data_np = scaler.fit_transform(all_data_np) # (N_days, 50)

scaler_path = "unified_scaler.pkl"
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"'{scaler_path}' 저장 완료.")

# (참고) 50개 컬럼의 순서 저장
column_map_path = "unified_column_map.pkl"
with open(column_map_path, 'wb') as f:
    pickle.dump(column_names, f)
print(f"'{column_map_path}' (컬럼 순서) 저장 완료.")

# --- 3. [수정] 단일 윈도우 생성 (50개 변수) ---
print("--- 슬라이딩 윈도우 생성 (50 features) ---")
inputs, outputs = [], []
total_len = len(scaled_data_np)
for i in range(total_len - input_len - output_len + 1):
    inputs.append(scaled_data_np[i : i + input_len, :])
    outputs.append(scaled_data_np[i + input_len : i + input_len + output_len, :])

inputs = torch.tensor(np.array(inputs), dtype=torch.float32)
outputs = torch.tensor(np.array(outputs), dtype=torch.float32)

dataset = TensorDataset(inputs, outputs)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print(f"Inputs shape: {inputs.shape}, Outputs shape: {outputs.shape}")

# --- 4. [수정] 단일 통합 모델 훈련 ---
print("--- 단일 통합 모델 훈련 ---")
model = iTransformer(
    input_len=input_len, 
    output_len=output_len, 
    num_features=num_features, # 50
    hidden_dim=hidden_dim, 
    num_heads=num_heads, 
    num_layers=num_layers
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs_batch, labels_batch in data_loader:
        inputs_batch, labels_batch = inputs_batch.to(device), labels_batch.to(device)
        
        optimizer.zero_grad()
        outputs_batch = model(inputs_batch) # (Batch, 14, 50)
        loss = criterion(outputs_batch, labels_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(data_loader):.6f}')

# --- 5. [수정] 단일 모델 저장 ---
model_path = "unified_model.pt"
torch.save(model.state_dict(), model_path)
print(f"'{model_path}' 저장 완료.")

Using device: cpu
--- 모든 주식 데이터 통합 ---
통합 데이터 형태: (1256, 50)
--- 단일 통합 스케일러 훈련 ---
'unified_scaler.pkl' 저장 완료.
'unified_column_map.pkl' (컬럼 순서) 저장 완료.
--- 슬라이딩 윈도우 생성 (50 features) ---
Inputs shape: torch.Size([219, 1024, 50]), Outputs shape: torch.Size([219, 14, 50])
--- 단일 통합 모델 훈련 ---
Epoch [10/100], Loss: 0.136936
Epoch [20/100], Loss: 0.095452
Epoch [30/100], Loss: 0.071136
Epoch [40/100], Loss: 0.054976
Epoch [50/100], Loss: 0.042844
Epoch [60/100], Loss: 0.034745
Epoch [70/100], Loss: 0.027922
Epoch [80/100], Loss: 0.022842
Epoch [90/100], Loss: 0.019123
Epoch [100/100], Loss: 0.015672
'unified_model.pt' 저장 완료.


Import Data

Training Implementation

Usage

In [10]:
csv_files = [
    'BROADCOM 5년치.csv', 'ALPHABET C 5년치.csv', 'AMAZON 5년치.csv', 
    'APPLE 5년치.csv', 'META 5년치.csv', 'MICROSOFT 5년치.csv', 
    'NETFLIX 5년치.csv', 'NVIDIA 5년치.csv', 'PALANTIR 5년치.csv', 'TESLA 5년치.csv'
]
stock_names = [re.match(r'^[A-Z]+', f).group(0) for f in csv_files]
feature_cols = ['Close/Last', 'Volume', 'Open', 'High', 'Low']
num_features = len(csv_files) * len(feature_cols) # 50

input_len = 1024
output_len = 14
# [!!!] hidden_dim을 128로 수정 (오류 해결)
hidden_dim = 128

model_path = "unified_model.pt"
scaler_path = "unified_scaler.pkl"
column_map_path = "unified_column_map.pkl"

device = torch.device('cpu')

# --- 2. 모델, 스케일러, 컬럼 맵 로드 ---
print("--- 모델(hidden_dim=128) 및 스케일러 로드 ---")
model = iTransformer(
    input_len=input_len, 
    output_len=output_len, 
    num_features=num_features, # 50
    hidden_dim=hidden_dim, # 128
    num_heads=8, 
    num_layers=6
).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

with open(scaler_path, 'rb') as f:
    scaler = pickle.load(f)
with open(column_map_path, 'rb') as f:
    column_names = pickle.load(f) # 50개 컬럼 이름 리스트

# --- 3. 추론용 데이터 준비 (훈련 시와 동일하게) ---
# ... (이전과 동일한 데이터 준비 코드) ...
print("--- 추론용 데이터 준비 ---")
all_data_dfs = []
for csv_file in csv_files:
    stock_name = re.match(r'^[A-Z]+', csv_file).group(0)
    df = pd.read_csv(csv_file, parse_dates=['Date'], index_col='Date')
    df = df.sort_index(ascending=True)
    
    cleaned_cols = {}
    for col in feature_cols:
        series = df[col]
        if series.dtype == 'object':
            series = series.str.replace(r'[$,]', '', regex=True)
        series = pd.to_numeric(series, errors='coerce')
        cleaned_cols[f"{stock_name}_{col}"] = series
    all_data_dfs.append(pd.DataFrame(cleaned_cols))

all_data_wide_df = pd.concat(all_data_dfs, axis=1)
all_data_wide_df = all_data_wide_df[column_names]
all_data_wide_df = all_data_wide_df.dropna()
all_data_np = all_data_wide_df.to_numpy(dtype=np.float32)
scaled_all_data_np = scaler.transform(all_data_np)
backtest_input = scaled_all_data_np[-input_len:] # (1024, 50)
new_input_tensor = torch.tensor(backtest_input, dtype=torch.float32).unsqueeze(0).to(device)

# --- 4. 예측 및 역정규화 ---
print("--- 예측 수행 ---")
with torch.no_grad():
    scaled_output = model(new_input_tensor) # (1, 14, 50)

original_scale_output = scaler.inverse_transform(scaled_output.squeeze(0).cpu().numpy())
original_input_data = scaler.inverse_transform(backtest_input)

# --- 5. 기대수익률 (mu) 추출 ---
print("--- 1. 기대수익률 (mu) 계산 ---")
predicted_mus = {}

for stock_name in stock_names:
    close_col_name = f"{stock_name}_Close/Last"
    close_index = column_names.index(close_col_name)
    current_price = original_input_data[-1, close_index]
    predicted_future_price = original_scale_output[-1, close_index]

    ratio_14day = (predicted_future_price / current_price) - 1
    annualized_mu = ((1 + ratio_14day) ** (252.0 / output_len)) - 1
    
    print(f"  [{stock_name}] 현재가: {current_price:.2f}, 14일 후 예측: {predicted_future_price:.2f} -> 연 {annualized_mu*100:.2f}%")
    predicted_mus[stock_name] = annualized_mu

# --- 6. 위험 (S) 계산 ---
print("--- 2. 위험 공분산 (S) 계산 (과거 종가 데이터 사용) ---")
close_price_df = pd.DataFrame()
for stock_name in stock_names:
    close_col_name = f"{stock_name}_Close/Last"
    close_price_df[stock_name] = all_data_wide_df[close_col_name]

S = risk_models.sample_cov(close_price_df, frequency=252)
mu_series = pd.Series(predicted_mus)

# --- 7. 포트폴리오 최적화 (L2_reg 헷징) ---
print("--- 3. 최적 포트폴리오 계산 (L2_reg gamma 헷징) ---")
ef = EfficientFrontier(mu_series, S)

# [!!!] 헷징 방식 수정: L2 정규화를 '목표'로 추가 (gamma=1.0)
# gamma 값을 높일수록 분산 효과(헷징)가 강해집니다.
ef.add_objective(objective_functions.L2_reg, gamma=10.0)

try:
    weights = ef.max_sharpe(risk_free_rate=0.02)
    cleaned_weights = ef.clean_weights()

    print("\n최적의 포트폴리오 비율 (L2_reg 헷징 적용):")
    weights_df = pd.DataFrame.from_dict(cleaned_weights, orient='index', columns=['Weight'])
    weights_df['Weight (%)'] = weights_df['Weight'].apply(lambda x: f"{x*100:.2f}%")
    print(weights_df)

    print("\n예상 포트폴리오 성과 (통합 모델 예측 mu 기준):")
    ef.portfolio_performance(verbose=True, risk_free_rate=0.02)

except Exception as e:
    print(f"최적화 오류: {e}")

--- 모델(hidden_dim=128) 및 스케일러 로드 ---
--- 추론용 데이터 준비 ---
--- 예측 수행 ---
--- 1. 기대수익률 (mu) 계산 ---
  [BROADCOM] 현재가: 369.63, 14일 후 예측: 353.10 -> 연 -56.10%
  [ALPHABET] 현재가: 281.82, 14일 후 예측: 256.73 -> 연 -81.33%
  [AMAZON] 현재가: 244.22, 14일 후 예측: 220.37 -> 연 -84.27%
  [APPLE] 현재가: 270.37, 14일 후 예측: 268.26 -> 연 -13.18%
  [META] 현재가: 648.35, 14일 후 예측: 715.90 -> 연 495.28%
  [MICROSOFT] 현재가: 517.81, 14일 후 예측: 509.74 -> 연 -24.64%
  [NETFLIX] 현재가: 1118.86, 14일 후 예측: 1156.65 -> 연 81.82%
  [NVIDIA] 현재가: 202.49, 14일 후 예측: 197.06 -> 연 -38.67%
  [PALANTIR] 현재가: 200.47, 14일 후 예측: 180.05 -> 연 -85.53%
  [TESLA] 현재가: 456.56, 14일 후 예측: 449.79 -> 연 -23.57%
--- 2. 위험 공분산 (S) 계산 (과거 종가 데이터 사용) ---
--- 3. 최적 포트폴리오 계산 (L2_reg gamma 헷징) ---

최적의 포트폴리오 비율 (L2_reg 헷징 적용):
            Weight Weight (%)
BROADCOM   0.00000      0.00%
ALPHABET   0.00000      0.00%
AMAZON     0.00000      0.00%
APPLE      0.00000      0.00%
META       0.86699     86.70%
MICROSOFT  0.00000      0.00%
NETFLIX    0.13301     13.30%
NVIDIA 

/opt/anaconda3/envs/quant/lib/python3.8/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
